In [1]:
import streamlit as st
import pandas as pd 
from streamlit_option_menu import option_menu
from cgitb import text
from multiprocessing import Value
from os import write
from turtle import onclick, onscreenclick
from typing import List
from numpy.core.fromnumeric import size
from PIL import Image
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json
from pathlib import Path


def cql_to_pandas(cql_query,cassandra_session):
    def pandaspark_factory(colnames, rows):\
        return pd.DataFrame(rows, columns=colnames)
    cassandra_session.row_factory = pandaspark_factory
    cassandra_session.default_fetch_size = None
    result = cassandra_session.execute(cql_query, timeout=None)
    return result._current_rows


cloud_config= {'secure_connect_bundle': r'secure-connect-henry.zip'}
auth_provider = PlainTextAuthProvider(json.load(open(r'log_in.json'))['log_user'], json.load(open(r'log_in.json'))['log_password'])
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

2022-12-01 04:34:11.755 WARNING cassandra.cluster: Downgrading core protocol version from 66 to 65 for df90de2b-07fa-4569-9bdd-2becfb016fd7-us-east1.db.astra.datastax.com:29042:6157e7e6-fc87-4471-8436-a55791b5c687. To avoid this, it is best practice to explicitly set Cluster(protocol_version) to the version supported by your cluster. http://datastax.github.io/python-driver/api/cassandra/cluster.html#cassandra.cluster.Cluster.protocol_version
2022-12-01 04:34:13.139 WARNING cassandra.cluster: Downgrading core protocol version from 65 to 5 for df90de2b-07fa-4569-9bdd-2becfb016fd7-us-east1.db.astra.datastax.com:29042:6157e7e6-fc87-4471-8436-a55791b5c687. To avoid this, it is best practice to explicitly set Cluster(protocol_version) to the version supported by your cluster. http://datastax.github.io/python-driver/api/cassandra/cluster.html#cassandra.cluster.Cluster.protocol_version
2022-12-01 04:34:14.664 ERROR   cassandra.connection: Closing connection <AsyncoreConnection(2505994124400) d

In [33]:
print(cql_to_pandas("""SELECT * FROM yelp.review ALLOW FILTERING""".format('luGY5CJlb4CJ3LrBcSXMPA'),session).iloc[0])

review_id                                 8YLmnA2LuDj_XqEkc_GnzQ
business_id                               tYCok-NtWvg8_k7woeB83w
cool                                                           0
date                                                  2017-08-07
funny                                                          0
stars                                                        5.0
text           Love love love this place for HH n dinner is g...
useful                                                         0
user_id                                   4qEBMFM2SiGe3oHCRfyNwQ
Name: 0, dtype: object


In [34]:
pd.read_csv(r'data/review_1000.csv')

,business_id,neg_reviews,pos_reviews
0,---kPU91CF4Lq2-WlRu9Lw,4,20
1,--LC8cIrALInl2vyo701tg,0,8
2,--epgcb7xHGuJ-4PUeSLAw,22,16
3,--gJkxbsiSIwsQKbiwm_Ng,0,6
4,--hF_3v1JmU9nlu4zfXJ8Q,0,15
...,...,...,...
995,1T4i26dJimZ0pB6x68Sk5g,0,7
996,1T84BU5oHZA4yDrKtucGVg,11,8
997,1T9F7iLDFAYWJvif1RKneg,83,359
998,1TAHFIs5ETVVB9X9R3Jwrw,0,5


In [41]:
cql_to_pandas("SELECT * FROM system_schema.tables WHERE keyspace_name = 'yelp'",session)

,keyspace_name,table_name,additional_write_policy,bloom_filter_fp_chance,caching,cdc,comment,compaction,compression,crc_check_chance,...,flags,gc_grace_seconds,id,max_index_interval,memtable_flush_period_in_ms,min_index_interval,nodesync,read_repair,read_repair_chance,speculative_retry
0,yelp,business,99PERCENTILE,0.01,"[keys, rows_per_partition]",None,,[class],"[chunk_length_in_kb, class]",1.0,...,(compound),864000,64a4a920-7118-11ed-a279-ad4a188271fa,2048,0,128,None,BLOCKING,0.0,99PERCENTILE
1,yelp,checkin,99PERCENTILE,0.01,"[keys, rows_per_partition]",None,,[class],"[chunk_length_in_kb, class]",1.0,...,(compound),864000,34cedf40-7118-11ed-a279-ad4a188271fa,2048,0,128,None,BLOCKING,0.0,99PERCENTILE
2,yelp,checkin_full,99PERCENTILE,0.01,"[keys, rows_per_partition]",None,,[class],"[chunk_length_in_kb, class]",1.0,...,(compound),864000,de995b20-712e-11ed-90c9-3520dc2fde2c,2048,0,128,None,BLOCKING,0.0,99PERCENTILE
3,yelp,review,99PERCENTILE,0.01,"[keys, rows_per_partition]",None,,[class],"[chunk_length_in_kb, class]",1.0,...,(compound),864000,f5e06850-7115-11ed-90c9-3520dc2fde2c,2048,0,128,None,BLOCKING,0.0,99PERCENTILE
4,yelp,tip,99PERCENTILE,0.01,"[keys, rows_per_partition]",None,,[class],"[chunk_length_in_kb, class]",1.0,...,(compound),864000,90d5aac0-7114-11ed-a279-ad4a188271fa,2048,0,128,None,BLOCKING,0.0,99PERCENTILE
5,yelp,top_tips,99PERCENTILE,0.01,"[keys, rows_per_partition]",None,,[class],"[chunk_length_in_kb, class]",1.0,...,(compound),864000,9a908630-7118-11ed-90c9-3520dc2fde2c,2048,0,128,None,BLOCKING,0.0,99PERCENTILE
6,yelp,user,99PERCENTILE,0.01,"[keys, rows_per_partition]",None,,[class],"[chunk_length_in_kb, class]",1.0,...,(compound),864000,f4ed91f0-7117-11ed-a279-ad4a188271fa,2048,0,128,None,BLOCKING,0.0,99PERCENTILE
7,yelp,user_full,99PERCENTILE,0.01,"[keys, rows_per_partition]",None,,[class],"[chunk_length_in_kb, class]",1.0,...,(compound),864000,b5bb6800-7125-11ed-90c9-3520dc2fde2c,2048,0,128,None,BLOCKING,0.0,99PERCENTILE
8,yelp,user_metrics,99PERCENTILE,0.01,"[keys, rows_per_partition]",None,,[class],"[chunk_length_in_kb, class]",1.0,...,(compound),864000,90ba7a40-7117-11ed-a279-ad4a188271fa,2048,0,128,None,BLOCKING,0.0,99PERCENTILE
9,yelp,user_metrics_full,99PERCENTILE,0.01,"[keys, rows_per_partition]",None,,[class],"[chunk_length_in_kb, class]",1.0,...,(compound),864000,25e326e0-711c-11ed-90c9-3520dc2fde2c,2048,0,128,None,BLOCKING,0.0,99PERCENTILE


In [40]:
pd.DataFrame(session.execute("SELECT * FROM system_schema.tables WHERE keyspace_name = 'yelp'"))

,0
0,keyspace_name
1,table_name
2,additional_write_policy
3,bloom_filter_fp_chance
4,caching
5,cdc
6,comment
7,compaction
8,compression
9,crc_check_chance
